In [ ]:
#--------------------------DNN TUNNING with Focal Loss and Class Weights--------------------------

!pip install tensorflow

# ----------------------- IMPORTS -----------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# ------------------- FOCAL LOSS -------------------
def sparse_categorical_focal_loss(gamma=2.0, alpha=0.25):
    def loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true_one_hot = tf.one_hot(y_true, depth=y_pred.shape[-1])
        cross_entropy = K.categorical_crossentropy(y_true_one_hot, y_pred)
        pt = tf.reduce_sum(y_true_one_hot * y_pred, axis=-1)
        focal_loss = alpha * tf.pow(1. - pt, gamma) * cross_entropy
        return focal_loss
    return loss_fn

# ------------------- DATA LOADING -------------------
attack_free = pd.read_csv("Attack_free new.csv")[0:2369397]
dos = pd.read_csv("DoS_Attack_new.csv")[0:656578]
fuzzy = pd.read_csv("Fuzzy_Attack_New.csv")[0:591989]
impersonation = pd.read_csv("Impersonation_Attack_New.csv")[0:995471]

# ------------------- Pre Processing -------------------
attack_free['label'] = 0
dos['label'] = 1
fuzzy['label'] = 2
impersonation['label'] = 3

df = pd.concat([attack_free, dos, fuzzy, impersonation], ignore_index=True)
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

for col in df.columns[:-1]:
    df[col] = df[col].apply(lambda x: int(str(x), 16) if isinstance(x, str) and all(c in '0123456789abcdefABCDEF' for c in str(x)) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(inplace=True)

X = df.drop(columns='label').values
y = df['label'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

# ------------------- CLASS WEIGHTS -------------------
cw = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(cw))

# ------------------- MODEL BUILDING -------------------
model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(448, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(96, activation='relu'),
    Dropout(0.4),
    Dense(48, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=sparse_categorical_focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy']
)

# ------------------- TRAINING -------------------
start_train = time.time()
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=256,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)
end_train = time.time()
print(f"\n Total Training Time: {end_train - start_train:.2f} seconds")

history_dnn = history


# === Plot Training & Validation Loss ===
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss', linestyle='-', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', linestyle='--', color='orange')
plt.title('Training vs Validation Loss (DNN)', fontsize=14, weight='bold')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.tight_layout()
plt.savefig("dnn_training_validation_loss.png", dpi=300)
plt.show()


# ------------------- EVALUATION -------------------
start_test = time.time()
loss, acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {acc:.4f}")
print(f" Loss: {loss:.4f}")
end_test = time.time()
print(f" Total Testing Time: {end_test - start_test:.2f} seconds")


#-------------------- PREDICTIONS AND REPORTS -------------------
y_pred = model.predict(X_test).argmax(axis=1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['AttackFree', 'DoS', 'Fuzzy', 'Impersonation'],
            yticklabels=['AttackFree', 'DoS', 'Fuzzy', 'Impersonation'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix Tuned DNN")
plt.tight_layout()
plt.savefig("confusion_matrix_final.png")

# ------------------- SAVE MODEL -------------------
model.save("final_CAN_IDS_model.h5")



# === Latency Measurement ===
sample_input = X_test[:1000]  # Use 1000 samples
start = time.time()
_ = model.predict(sample_input, verbose=0)
end = time.time()

latency = (end - start) / len(sample_input)
print(f"\nInference Time for 1000 samples: {end - start:.4f} seconds")
print(f"Average Latency per Sample: {latency * 1000:.4f} ms")



# === TPR & TNR Calculation for DNN ===

TPR = []  # True Positive Rate (Recall)
TNR = []  # True Negative Rate (Specificity)

for i in range(len(cm)):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)

    tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) != 0 else 0

    TPR.append(tpr)
    TNR.append(tnr)

# === Print TPR and TNR Table ===
print("\n True Positive Rate (TPR) and True Negative Rate (TNR) for each class:")
for i, cls in enumerate(['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']):
    print(f"{cls}:")
    print(f"   TPR (Recall): {TPR[i]:.4f}")
    print(f"   TNR (Specificity): {TNR[i]:.4f}")


# === TPR & FPR Calculation for DNN ===
TPR = []  
FPR = []  

for i in range(len(cm)):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)

    tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) != 0 else 0

    TPR.append(tpr)
    FPR.append(fpr)

# === Print TPR and FPR Table ===
print("\ True Positive Rate (TPR) and False Positive Rate (FPR) for each class:")
for i, cls in enumerate(['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']):
    print(f"{cls}:")
    print(f"   TPR (Recall): {TPR[i]:.4f}")
    print(f"   FPR (Fall-out): {FPR[i]:.4f}")


# === Average TPR and FPR ===
avg_tpr = np.mean(TPR)
avg_fpr = np.mean(FPR)
# === Overall accuracy
overall_accuracy = accuracy_score(y_test, y_pred)

# === Print Summary ===
print("\n Average Metrics Across All Datasets:")
print(f" Average TPR (Recall): {avg_tpr:.4f}")
print(f" Average FPR (Fall-out): {avg_fpr:.4f}")
print(f" Overall Accuracy: {overall_accuracy:.4f}")
